In [16]:
import pandas as pd
import numpy as np 
import seaborn as sn
import matplotlib.pyplot as plt
df = pd.read_csv('winequalityN.csv')


Olhando abaixo, tem muitos valores N/A. Vamos substituir eles pela média.

In [18]:
df.isna().sum()

type                     0
fixed acidity           10
volatile acidity         8
citric acid              3
residual sugar           2
chlorides                2
free sulfur dioxide      0
total sulfur dioxide     0
density                  0
pH                       9
sulphates                4
alcohol                  0
quality                  0
dtype: int64

In [22]:
df['fixed acidity'].fillna(df['fixed acidity'].median(), inplace=True)
df['volatile acidity'].fillna(df['volatile acidity'].mean(), inplace=True)
df['citric acid'].fillna(df['citric acid'].mean(), inplace=True)
df['residual sugar'].fillna(df['residual sugar'].mean(), inplace=True)
df['chlorides'].fillna(df['chlorides'].mean(), inplace=True)
df['pH'].fillna(df['pH'].mean(), inplace=True)
df['sulphates'].fillna(df['sulphates'].mean(), inplace=True)

Depois de converter para a média não tem mais N/A:

In [23]:
df.isna().sum()

type                    0
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [24]:
df['quality'].value_counts()

6    2836
5    2138
7    1079
4     216
8     193
3      30
9       5
Name: quality, dtype: int64

In [1]:
sns.set_theme(style="whitegrid"
ax = sb.boxplot(x=df['citric acid'])

SyntaxError: invalid syntax (3954890542.py, line 2)